In [ ]:
%qtconsole

# Expectation Maximization (theory)
Please see slides 14-15 in `ML-Clustering-L2.pdf` for an explanation of the Expecation Maximization algorithm. Let $D = \{x_1,...,x_n\}\subseteq \mathbb{R}^d$. Recall that in the EM algorithm we represent a cluster $C_i$ by a Gaussian distribution whose density function is given by

$$
\Pr(x|C_i) = \frac{1}{\sqrt{(2\pi)^d|\Sigma_i|}}\cdot e^{-\frac{1}{2}\cdot (x-\mu_i)^\intercal\cdot (\Sigma_i)^{-1}\cdot (x-\mu_i)}.
$$

Each cluster $C_i$ is parameterized with a cluster mean $\mu_i\in\Bbb R^d$, a covariance matrix $\Sigma_i\in\Bbb R^{d\times d}$ and a prior probability $\Pr(C_i)$.

<!--Given a clustering $M = \{C_1,\ldots,C_k\}$, we can compute
$$
\Pr(x) = \sum_{i=1}^k \Pr(C_i)\cdot\Pr(x|C_i),
$$
where we estimate $\Pr(C_i)$ by $W_i$, the frequency of the cluster $C_i$ in the data (i.e. the ratio of data points belonging to cluster $C_i$).-->

The Expectation Maximization has two steps: Expectation and Maximization (hence the name). Before this step we need to initialize an initial clustering $M = \{C_1,\ldots,C_k\}$. This is done as follows: 
- Choose cluster mean $\mu_i\in\Bbb R^d$ uniformly at random (in the adequate region). 
- Initialize the covariance matrix $\Sigma_i\in\Bbb R^{d\times d}$ as the identity matrix. 
- Initialize the prior probability $\Pr(C_i)$ is to $\frac{1}{k}$. 


Then we repeat the following two steps until the sum $\sum_{i=1}^k \|\mu_i - \mu'_i\|$ is below some pre-specified prespecified threshold $\epsilon$ (where $\mu'_i$ and $\mu_i$ are the means computed in two consecutive executions).

- <b>Update probabilities (expectation step):</b> For each all pairs $C_i$ and $x_j$ compute $\Pr(C_i|x_j) = \frac{\Pr(x_j|C_i)\cdot \Pr(C_i)}{\Pr(x_j)}$. <br>
  To do this we need to compute $\Pr(x_j|C_i)$ and $\Pr(x_j)$. Compute $\Pr(x_j|C_i)$ by the formula above, and compute $\Pr(x)$ by $\Pr(x)= \sum_{i=1}^k \Pr(C_i)\cdot\Pr(x|C_i)$.<br><br>
  
- <b>Update clustering (maximization step): </b>Compute a new model $M = \{C_1,\ldots, C_k\}$ by recomputing $W_i$, $\mu_i$ and $\Sigma_i$ as

$$
W_i = \frac{1}{n}\cdot\sum_{x\in D}\Pr(C_i|x)\approx P(C_i),
\qquad
\mu_i = \frac{\sum_{x\in D}^n x\Pr(C_i|x)}{\sum_{x\in D} \Pr(C_i|x)},
\qquad
\Sigma_i = \frac{\sum_{x\in D} \Pr(C_i|x)\cdot(x-\mu_i)\cdot(x-\mu_i)^\intercal}{\sum_{x\in D} \Pr(C_i|x)}.
$$

<b>Question 1:</b> What is the objective of Expectation Maximization (EM) clustering problem? 

HINT: See slides pages 14-15. 

<!--<b>Question 1 (b): </b>Rewrite the objective for $\text{dist}(x,y)=||x-y||$-->

<b>Question 2:</b> What are the E-step, M-step? What is estimated, what is fixed?

<b>Question 3:</b> Does EM algorithm provide any guarantees for finding the optimal solution?

HINT: What is the optimal solution? 

<b>Question 4:</b> Can a cluster of a EM clustering be empty? 

HINT: Since we are dealing with probabilities, what would it mean for a cluster to be 'empty'?

<b>Question 5:</b> In which of the two step does the EM algorithm attempt to maximize the objective function?

<b>Question 6:</b> Is EM algorithm guaranteed to converge? 

HINT: Assume the objective is strictly decreasing. 

<b>Question 7 (a): </b>How is Expectation Maximization Clustering a generalizaton of k-means clustering?

HINT: See the book [ZM] page 353. 

<b>Question 7 (b): </b>Could we assign a "hard clustering" of all points instead of a probability?

HINT: Which cluster is it most likely that $x_i$ belongs to?

<b>Question 7 (c): </b>Both models rely on certain assumptions on the data. What are these assumptions? Which assumptions are shared?

<b>Question 7 (d): </b>Give examples of data where the assumptions of k-means clustering and EM clustering are reasonable/unreasonable. (relevant for the exam)

<b>Bonus Question (Hard): </b>Baum-Welch training of Hidden Markov Models was also called Expectation Maximization. Are there any similarities/differences between the EM algorithm above and the Baum-Welch EM algorithm?

<!--
TODO: Some question that I thought about which would be nice to have is like a discussion about the differences between Kmeans and EM. The book points out (page 353) that EM is some kind of generalization of Kmeans, so in some sense it's more versatil (for example, it doesn't restrict us to convex clusters -right?- and it seems like the fact that the clusters are ''randomized'' allows us to do more things). What are the advantages/disadvantages of each? -->

# Expectation Maximization (code)
In this exercise you must implement the EM algorithm. To test our implementation we will need data. Like last week we use the Iris dataset. Recall that the dataset has three clases so we <i>cheat</i> by setting $k=3$.

In [ ]:
# Load the Iris data set
import sklearn.datasets
iris = sklearn.datasets.load_iris()
X = iris['data'][:,0:2] # reduce dimensions so we can plot what happens.
k = 3
print(X.shape)

## Implementing EM algorithm
Remember the Expectation Maximization algorithm has two steps. Let us first implement the Expectation step. For this step it suffices to compute $\Pr(C_i\mid x_j)$ for $i=1,\ldots,k$ and $j=1,\ldots n$. Remember that $\Pr(C_i \mid x_j)$ can be computed as 

$$\Pr(C_i|x_j) = \frac{\Pr(x_j|C_i)\cdot \Pr(C_i)}{\Pr(x_j)}$$

The first goal will then be to compute each part of the above equation. To do this we will need the parameters: ($\mu_i, \Sigma_i, \Pr(C_i)$). These will be represented in Python as follows: 

$$\text{means}=\begin{pmatrix}
- & \mu_1^T & - \\
- & \vdots & - \\
- & \mu_k^T & - 
\end{pmatrix}\in \mathbb{R}^{k\times d} \quad\quad
\text{probs_c}=\begin{pmatrix}
\Pr(C_1)\\
\vdots \\
\Pr(C_k)
\end{pmatrix}\in \mathbb{R}^{k}
$$

Similarly we represent the $\Sigma_i$'s as $\text{covs}\in\mathbb{R}^{k\times d \times d}$ such that $\text{covs[i]}=\Sigma_i$. Finally we represent $\Pr(x)$'s and $\Pr(C_i \mid x)$ as 

$$\text{prob_x}=\begin{pmatrix}
\Pr(x_1) \\
\vdots \\
\Pr(x_n) 
\end{pmatrix}\in \mathbb{R}^{k\times d} \quad\quad
\text{probs_cx}=\begin{pmatrix}
\Pr(C_1\mid x_1) & \ldots & \Pr(C_1 \mid x_n)\\
\vdots \\
\Pr(C_k \mid x_1) & \ldots & \Pr(C_k \mid x_n)
\end{pmatrix}\in \mathbb{R}^{k}
$$

The function *compute_probs_cs* below takes `means`, `probs_c` and `covs` as input and returns `prob_cx` and `prob_x`. 

<b>Question: </b>What is the dimensions of `prob_cx` and how can we compute it given `means`, `probs_c` and `covs`?

<!-- The algorithm returns a clustering $M = \{C_1,\ldots,C_k\}$. This corresponds to a 
 - sequence of means $(\mu_1,\ldots,\mu_k)$ where each $\mu_i\in\Bbb R^d$,
 - sequence of covariance matrices $(\Sigma_1,\ldots,\Sigma_k)$ where each $\Sigma_i\in\Bbb R^{d\times d}$,
 - prior probabilities $(\Pr(C_1),\ldots,\Pr(C_k))$. 

Given $x$ we can then compute the probability of $x$ conditioned on the $i$'th cluster $Pr(C\mid x_i)$. If we want to assign $x$ to a specific cluster we compute $(\Pr(x|C_1),\ldots,\Pr(x|C_k))$ and assign $x$ to the `arg max`.-->

<!--The following helper function takes a description of a Gaussian Mixture ($\mu_i$'s, $\Sigma_i$'s and $\Pr(C_i)$'s)) and returns the probability densities of each point. We represent the inputs as 

If you want more information you can take a look at scipy's [multivariate_normal](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.multivariate_normal.html) documentation.-->

In [ ]:
import numpy as np
from scipy.stats import multivariate_normal

def compute_probs_cx(points, means, covs, probs_c):
    '''
    Input
      - points: (n times d) array containing the dataset
      - means:  (k times d) array containing the k means
      - covs:   (k times d times d) array such that cov[j,:,:] is the covariance matrix of the j-th Gaussian.
      - priors: (k) array containing priors
    Output
      - probs:  (k times n) array such that the entry (i,j) represents Pr(C_i|x_j)
    '''
    # Convert to numpy arrays.
    points, means, covs, probs_c = np.asarray(points), np.asarray(means), np.asarray(covs), np.asarray(probs_c)
    
    # Get sizes
    n, d = points.shape
    k = means.shape[0]
    
    # Compute probabilities
    # This will be a (k, n) matrix where the (i,j)'th entry is Pr(C_i)*Pr(x_j|C_i).
    probs_cx = np.zeros((k, n))
    for i in range(k):
        try:
            probs_cx[i] = probs_c[i] * multivariate_normal.pdf(mean=means[i], cov=covs[i], x=points)
        except Exception as e:
            print("Cov matrix got singular: ", e)
    
    # The sum of the j'th column of this matrix is P(x_j); why?
    probs_x = np.sum(probs_cx, axis=0, keepdims=True) 
    assert probs_x.shape == (1, n)
    
    # Divide the j'th column by P(x_j). The the (i,j)'th then 
    # becomes Pr(C_i)*Pr(x_j)|C_i)/Pr(x_j) = Pr(C_i|x_j)
    probs_cx = probs_cx / probs_x
    
    return probs_cx, probs_x

The following is the basic structure for the EM algorithm. For the Expectation step it calls the above code. You need to finish the Maximization step 

<b>Question: </b>Where in the code (and how) do we initialize `means`, `covs` and `probs_c`?

<b>Question: </b>Which three things do you need to compute in the Maximization step?

In [ ]:
def em_algorithm(X, k, T, epsilon = 0.001, means=None):
    """ Clusters the data X into k clusters using the Expectation Maximization algorithm. 
    
        Parameters
        ----------
        X : Data matrix of shape (n, d)
        k : Number of clusters.
        T : Maximum number of iterations
        epsilon :  Stopping criteria for the EM algorithm. Stops if the means of
                   two consequtive iterations are less than epsilon.
        means : (k times d) array containing the k initial means (optional)
        
        Returns
        -------
        means:     (k, d) array containing the k means
        covs:      (k, d, d) array such that cov[j,:,:] is the covariance matrix of 
                   the Gaussian of the j-th cluster
        probs_c:   (k, ) containing the probability Pr[C_i] for i=0,...,k. 
        llh:       The log-likelihood of the clustering (this is the objective we want to maximize)
    """
    n, d = X.shape
    
    # Initialize and validate mean
    if means is None: 
        means = np.random.rand(k, d)

    # Initialize cov, prior
    probs_x  = np.zeros(n) 
    probs_cx = np.zeros((k, n)) 
    probs_c  = np.zeros(k) + np.random.rand(k)
    
    covs = np.zeros((k, d, d))
    for i in range(k): covs[i] = np.identity(d)
    probs_c = np.ones(k) / k
    
    # Column names
    print("Iterations\tLLH")
    
    close = False
    old_means = np.zeros_like(means)
    iterations = 0
    while not(close) and iterations < T:
        old_means[:] = means 

        # Expectation step
        probs_cx, probs_x = compute_probs_cx(X, means, covs, probs_c)
        assert probs_cx.shape == (k, n)
        
        # Maximization step
        # YOUR CODE HERE
        # END CODE
        
        # Compute per-sample average log likelihood (llh) of this iteration     
        llh = 1/n*np.sum(np.log(probs_x))
        print(iterations+1, "\t\t", llh)

        # Stop condition
        dist = np.sqrt(((means - old_means) ** 2).sum(axis=1))
        close = np.all(dist < epsilon)
        iterations += 1
        
    # Validate output
    assert means.shape == (k, d)
    assert covs.shape == (k, d, d)
    assert probs_c.shape == (k,)
    return means, covs, probs_c, llh

# Load the Iris data set
import sklearn.datasets
iris = sklearn.datasets.load_iris()
X = iris['data'][:,0:2] # reduce dimensions so we can plot what happens.
k = 3

means, covs, priors, llh = em_algorithm(X, 3, 100, 0.001)

The random initialization usually causes the algorithm to get stuck at different local maximum. This causes different runs to get different scores. In practice this is usually handled by running the algorithm several times and picking the best run. 

The following code runs EM algorithm 50 times and plots the score of each run. Because the data set is fairly small, $n=150$, most of the runs will get the same score. 

In [ ]:
%matplotlib notebook 
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(8,3))    
llhs = []

for i in range(100):
    _, _, _, llh = em_algorithm(X, 3, 100)
    llhs.append(llh)
    ax.plot(llhs, 'bx')
    fig.canvas.draw() 

To check your implementation you should run <a href="http://scikit-learn.org/stable/modules/mixture.html">sklearn</a>'s implementation of the EM algorithm. You might want to take a look at the documentation to get a better understanding of what the algorithm is actually doing.

<!--By default the implementation repeats the algorithm $10$ times and picks the best result. A sanity check for your implementation of Lloyd's algorithm is to check that the scores are roughly the same. -->

In [ ]:
from sklearn.mixture import GaussianMixture as EM
expectation_maximization = EM(n_components=3, init_params='random', covariance_type='diag', verbose=2, verbose_interval =1).fit(X)

print(expectation_maximization.score(X))

To get a visual understanding of the algorithm, the following code visualizes each step of the algorithm. Similarly to previous week, it prints the centroids and the corresponding cluster each point is assinged to. Furthermore, because the clusters are represented as gaussians, the gaussians are plotted. They are first plotted by themselves, they are plotted together (by summing). 

To run the visualization you need to copy and paste your implementation from above. Then the code should run with no modifications. Running the visualization might be a bit slow; try change the `detail` parameter if you have any problems. 

In [ ]:
%matplotlib notebook 
import matplotlib.pyplot as plt
import numpy as np
import time
from mpl_toolkits.mplot3d import Axes3D

def em_algorithm_visualize(X, k, T, epsilon = 0.001, means=None, detail=20):
    """ Clusters the data X into k clusters using the Expectation Maximization algorithm. 
    
        Parameters
        ----------
        X : Data matrix of shape (n, d)
        k : Number of clusters.
        T : Maximum number of iterations
        epsilon :  Stopping criteria for the EM algorithm. Stops if the means of
                   two consequtive iterations are less than epsilon.
        means : (k times d) array containing the k initial means (optional)
        
        Returns
        -------
        means:     (k, d) array containing the k means
        covs:      (k, d, d) array such that cov[j,:,:] is the covariance matrix of 
                   the Gaussian of the j-th cluster
        probs_c:   (k, ) containing the probability Pr[C_i] for i=0,...,k. 
        llh:       The log-likelihood of the clustering (this is the objective we want to maximize)
    """
    n, d = X.shape
    
    # Visualization stuff. 
    fig, (ax, _, _, _, _) = plt.subplots(5, 1, figsize=(10,16)) 
    ax.axis('off')
    colors = ["r", "g", "b"]
    ax3d = fig.add_subplot(2, 1, 2, projection='3d')
    ax3d1 = fig.add_subplot(3, 1, 2, projection='3d')
    ax3d2 = fig.add_subplot(4, 1, 2, projection='3d')
    ax3d3 = fig.add_subplot(5, 1, 2, projection='3d')
    ax3d.set_axis_off()
    ax3d1.set_axis_off()
    ax3d2.set_axis_off()
    ax3d3.set_axis_off()
    
    # Initialize and validate mean
    if means is None: means = np.random.rand(k, d)

    # Initialize 
    probs_x  = np.zeros(n) 
    probs_cx = np.zeros((k, n)) 
    probs_c  = np.zeros(k) 
    covs = np.zeros((k, d, d))
    for i in range(k): covs[i] = np.identity(d)
    probs_c = np.ones(k) / k

    # END CODE
    
    # Column names
    print("Iterations\tLLH")
    
    close = False
    old_means = np.zeros_like(means)
    iterations = 0
    while not(close) and iterations < T:
        old_means[:] = means # This allows us to actually copy the array mean

        # Expectation step
        probs_cx, probs_x = compute_probs_cx(X, means, covs, probs_c)
        assert probs_cx.shape == (k, n)
        
        # Maximization step
        # YOUR CODE HERE
        # END CODE
        
        # Finish condition
        dist = np.sqrt(((means - old_means) ** 2).sum(axis=1))
        close = np.all(dist < epsilon)
        iterations += 1
        
        
        # !----------- VISUALIZATION CODE -----------!
        centroids = means
        # probs_cx's (i,j) is Pr[C_i, x_j]
        # assign each x_i to the cluster C_i that maximizes P(C_i | x_j)
        clustering = np.argmax(probs_cx, axis=0)
        assert clustering.shape == (n,), clustering.shape
        
        # Draw clusters
        ax.cla()
        for j in range(k):
            centroid = centroids[j]
            c = colors[j]
            ax.scatter(centroid[0], centroid[1], s=123, c=c, marker='^')
            data = X[clustering==j]
            x = data[:,0]
            y = data[:,1]
            ax.scatter(x, y, s=3, c=c)
            
        # draw 3d gaussians. 
        #Create grid and multivariate normal
        xs = np.linspace(4,7, 50)
        ys = np.linspace(2,4.5, 50)
        Xs, Ys = np.meshgrid(xs, ys)
        pos = np.empty(Xs.shape + (2,))
        pos[:, :, 0] = Xs; pos[:, :, 1] = Ys
        prob_space = sum([multivariate_normal(means[j], covs[j]).pdf(pos) for j in range(k)])

        #Make a 3D plot
        ax3d.cla()
        ax3d1.cla()
        ax3d2.cla()
        ax3d3.cla()
        ax3d.plot_surface(Xs, Ys, prob_space, cmap='viridis', linewidth=0)
        ax3d1.plot_surface(Xs, Ys, multivariate_normal(means[0], covs[0]).pdf(pos), cmap='viridis', linewidth=0)
        ax3d2.plot_surface(Xs, Ys, multivariate_normal(means[1], covs[1]).pdf(pos), cmap='viridis', linewidth=0)
        ax3d3.plot_surface(Xs, Ys, multivariate_normal(means[2], covs[2]).pdf(pos), cmap='viridis', linewidth=0)
        
        fig.canvas.draw()
        
    # Validate output
    assert means.shape == (k, d)
    assert covs.shape == (k, d, d)
    assert probs_c.shape == (k,)
    return means, covs, probs_c, llhs

# Load the Iris data set
import sklearn.datasets
iris = sklearn.datasets.load_iris()
X = iris['data'][:,0:2] # reduce dimensions so we can plot what happens.
k = 3

# the higher the detail the slower plotting
detail = 20 # 50 looks very nice but your computer might not be able to handle it. 
means, covs, priors, llh = em_algorithm_visualize(X, 3, 40, 0.001, detail=detail)

## Initializing EM with Lloyd's algorithm 

So far we have been initializing the means for the Expectation Maximization algorithm at random. We could also make a mix of Lloyd's algorithm and EM algorithm by running Lloyd's algorithm first to obtain the initial means for the EM algorithm. 

Begin by copying and pasting the implementation of Lloyd's algorithm from the previous week

In [ ]:
def lloyds_algorithm(X, k, T):
    """ Clusters the data of X into k clusters using T iterations of Lloyd's algorithm. 
    
        Parameters
        ----------
        X : Data matrix of shape (n, d)
        k : Number of clusters.
        T : Maximum number of iterations to run Lloyd's algorithm. 
        
        Returns
        -------
        clustering: A vector of shape (n, ) where the i'th entry holds the cluster of X[i].
        centroids:  The centroids/average points of each cluster. 
        cost:       The cost of the clustering 
    """
    pass

Then use this algorithm to initialize the means for the EM algorithm. For this notice that `em_algorithm` accepts an optional input for the initial means.

Also, notice that the sklearn's implementation of the EM algorithm can take this initialization into account. Can you look through the documentation and find out what lines should be changed when we used sklearn before? This would be very useful for testing and comparing your implementation above.

In [ ]:
# YOUR CODE HERE
# END CODE

<b>Question:</b> Why can we use Lloyd's algorithm to initialize the EM algorithm? Does it (always) give a better final clustering? If so, why is that the case?

## Evaluating the clustering using silhuette coefficient
In the lecture Ira talked about how one can compare different clusters.

In [ ]:
def silhouette(data, clustering): 
    n, d = data.shape
    k = np.unique(clustering)[-1]+1

    # YOUR CODE HERE
    silh = None
    # END CODE

    return silh

silhouette(X, clustering)

Calcualte the Silhouette coefficient for the EM clustering. 

# Clustering digits
In previous weeks we did supervised learning on images of digits. In this exercise we will perform clustering on digits. Remember clustering can be considered a type of unsupervised learning. The main difference to what we did before is that  will attempt to find patterns in the data without using the labels.  

You can use the AUDigits if you want. The following code uses a data set of images called MNIST. They are almost identical. The only reason for using MNIST is that we can import it with just two lines of code. 

In [ ]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("data/")

X = mnist.train.images
y = mnist.train.labels

print(X.shape)

The following code runs the Expectation Maimization algorithm on 5000 images from the MNIST dataset. It then visualizes the found centroids. 

In [ ]:
from sklearn.mixture import GaussianMixture as EM
import matplotlib.pyplot as plt

# One cluster for each digit
k = 10

# Run EM algorithm on 1000 images from the MNIST dataset. 

expectation_maximization = EM(n_components=k, max_iter=10, init_params='kmeans', covariance_type='diag', verbose=1, verbose_interval =1).fit(X)
print(expectation_maximization.score(X[:1000]))

means = expectation_maximization.means_
covs = expectation_maximization.covariances_
      
print(means.shape)
fig, ax = plt.subplots(1, k, figsize=(8, 1))

for i in range(k):
    ax[i].imshow(means[i].reshape(28, 28), cmap='gray')
    
plt.show()

<b>Question 1: </b>Why do the centroids look like images of digits? 
    
Because our clusters are represented as gaussians, it is possible to sample points from them. Think of this as rolling a dice, but instead of getting a number, you get a picture of a digit. The last line has a call to `sample`. The last parameter specifies which cluster to sample from. 

In [ ]:
from scipy.stats import multivariate_normal
import numpy as np

def sample(means, covs, num):
    mean = means[num]
    cov = covs[num]
     
    fig, ax = plt.subplots(1, 10, figsize=(8, 1))
    
    for i in range(10):
        img = multivariate_normal.rvs(mean=mean, cov=cov) # draw random sample   
        ax[i].imshow(img.reshape(28, 28), cmap='gray') # draw the random sample
    plt.show()